In [25]:
from transformers import BertTokenizer, BertModel
import numpy as np
import torch

# Import Packages
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
pd.set_option('display.max_colwidth', 100)
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
from gensim.models import KeyedVectors
import gensim.downloader
from collections import deque

In [26]:
# load csv from zip file
main_data = pd.read_csv('/Users/andrewsimon/Desktop/IMDBDataset.csv.zip')
main_data

,review,sentiment
0,One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked....,positive
1,A wonderful little production. <br /><br />The filming technique is very unassuming- very old-ti...,positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei off...",positive
...,...,...
49995,"I thought this movie did a down right good job. It wasn't as creative or original as the first, ...",positive
49996,"Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ...",negative
49997,"I am a Catholic taught in parochial elementary schools by nuns, taught by Jesuit priests in high...",negative
49998,I'm going to have to disagree with the previous comment and side with Maltin on this one. This i...,negative


In [27]:
# Map sentiment via one hot encoding
main_data['sentiment'] = main_data['sentiment'].map({'positive':1, 'negative': 0})
main_data

,review,sentiment
0,One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked....,1
1,A wonderful little production. <br /><br />The filming technique is very unassuming- very old-ti...,1
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",1
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his...,0
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei off...",1
...,...,...
49995,"I thought this movie did a down right good job. It wasn't as creative or original as the first, ...",1
49996,"Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ...",0
49997,"I am a Catholic taught in parochial elementary schools by nuns, taught by Jesuit priests in high...",0
49998,I'm going to have to disagree with the previous comment and side with Maltin on this one. This i...,0


In [28]:
toeknizer = BertTokenizer.from_pretrained('/Users/andrewsimon/bert-base-uncased')
model = BertModel.from_pretrained('/Users/andrewsimon/bert-base-uncased')

Some weights of the model checkpoint at /Users/andrewsimon/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
from transformers import AutoTokenizer, BertForPreTraining
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForPreTraining.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello", return_tensors="pt")
outputs = model(**inputs)

prediction_logits = outputs.prediction_logits
seq_relationship_logits = outputs.seq_relationship_logits

In [30]:
for i in range(len(main_data['review'])):
    try:
        inputs = tokenizer(main_data[i], return_tensors='pt')
        outputs = model(**inputs)
        prediction_logits = outputs.prediction_logits
        main_data['review'][i] = prediction_logits
    except KeyError:
        main_data['review'][i] = np.zeros(300,)

/var/folders/8f/mjj28lvs5fb1m_c_tvmtckl80000gn/T/ipykernel_31226/3692553745.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['review'][i] = np.zeros(300,)


In [31]:
# Separate into training and testing data
X_train, X_test, y_train, y_test = train_test_split(main_data['review'], main_data['sentiment'], test_size=0.2, random_state=1516)

In [32]:
X_train.shape

(40000,)

In [33]:
# Cast to numpy arrays
X_train = X_train.to_numpy()

In [34]:
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [35]:
training_padded = pad_sequences(X_train, maxlen=120, truncating='post')

In [36]:

testing_padded = pad_sequences(X_test, maxlen=120, truncating='post')

In [37]:
# Create model, evaluate

batch = 32

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', 'Recall', 'AUC', 'Precision', 'FalseNegatives', 'FalsePositives'])
    
num_epochs = 10
model.fit(training_padded, y_train, epochs=num_epochs,batch_size=batch, validation_data=(testing_padded, y_test))

Epoch 1/10


2023-05-09 23:01:12.022751: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1250/1250 [==============================] - 1s 779us/step - loss: 0.6932 - accuracy: 0.4963 - recall: 0.5019 - auc: 0.4985 - precision: 0.4959 - false_negatives: 9953.0000 - false_positives: 10194.0000 - val_loss: 0.6932 - val_accuracy: 0.4983 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_false_negatives: 5017.0000 - val_false_positives: 0.0000e+00
Epoch 2/10
1250/1250 [==============================] - 1s 561us/step - loss: 0.6932 - accuracy: 0.4962 - recall: 0.4794 - auc: 0.4989 - precision: 0.4956 - false_negatives: 10404.0000 - false_positives: 9749.0000 - val_loss: 0.6932 - val_accuracy: 0.4983 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_false_negatives: 5017.0000 - val_false_positives: 0.0000e+00
Epoch 3/10
1250/1250 [==============================] - 1s 602us/step - loss: 0.6932 - accuracy: 0.5003 - recall: 0.3106 - auc: 0.4987 - precision: 0.4998 - false_negatives: 13777.0000 - false_positives: 6210.0000 - val_los